In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305;

In [3]:
t = 1500; tt = 1500; tstep = 0.001; tspan = [time, tt]
integ_set = (alg = Vern9(), adaptive = true, abstol = 1e-15, reltol = 1e-15);

In [4]:
I0 = -1.702; U0 = 0.265
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [5]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = true, abstol = 1.0e-15, reltol = 1.0e-15)
 parameters:    [1.58, 0.013, 0.07993, 3.3, 3.07, 0.75, 0.4, 0.265, 0.305, 0.3, -1.702]
 time:          0.0
 state:         [11.325905642223786, 0.6594706953104683, 0.4863175026548461]


In [6]:
fp, ei, _ = fixedpoints(ds, box, jacob_TM_)

index_fp = 1
index_vec = 1
ϵ = 1e-9

Jac = jacob_TM_(fp[index_fp], p, 0)
eivecs = eigvecs(Jac)
shift =  fp[index_fp] + real(eivecs[:, index_vec])*ϵ
println("fixed point with shift along stable manifold: $(shift)")

fixed point with shift along stable manifold: [8.35127932322883, 0.7384633629560452, 0.4381456557568356]


In [39]:
ei[1]

3-element Vector{ComplexF64}:
 -6.408846522360701 + 0.0im
   2.14926644562012 - 13.807249654267821im
   2.14926644562012 + 13.807249654267821im

In [35]:
eivecs

3×3 SMatrix{3, 3, ComplexF64, 9} with indices SOneTo(3)×SOneTo(3):
   0.998722+0.0im     0.999855-0.0im            0.999855+0.0im
 -0.0491193+0.0im   -0.0141637-0.00933177im   -0.0141637+0.00933177im
  0.0119078+0.0im  0.000708384-0.00164424im  0.000708384+0.00164424im

In [37]:
eivecs[:, 1]

3-element SVector{3, ComplexF64} with indices SOneTo(3):
    0.9987219346344215 + 0.0im
 -0.049119256347767946 + 0.0im
  0.011907809870860554 + 0.0im

In [41]:
eigen_ = eigen(Jac)

Eigen{ComplexF64, ComplexF64, SMatrix{3, 3, ComplexF64, 9}, SVector{3, ComplexF64}}
values:
3-element SVector{3, ComplexF64} with indices SOneTo(3):
 -6.408846522360701 + 0.0im
   2.14926644562012 - 13.807249654267821im
   2.14926644562012 + 13.807249654267821im
vectors:
3×3 SMatrix{3, 3, ComplexF64, 9} with indices SOneTo(3)×SOneTo(3):
   0.998722+0.0im     0.999855-0.0im            0.999855+0.0im
 -0.0491193+0.0im   -0.0141637-0.00933177im   -0.0141637+0.00933177im
  0.0119078+0.0im  0.000708384-0.00164424im  0.000708384+0.00164424im

In [46]:
real(eigen_.vectors[:, 2])

3-element SVector{3, Float64} with indices SOneTo(3):
  0.9998545402866598
 -0.014163718853683456
  0.0007083836064604048

In [51]:
normalize(real(eigen_.vectors[:, 2]))

3-element SVector{3, Float64} with indices SOneTo(3):
  0.9998994295439241
 -0.014164354744999467
  0.0007084154099004763

In [57]:
normalize(imag(eigen_.vectors[:, 3]))

3-element SVector{3, Float64} with indices SOneTo(3):
 0.0
 0.9848294156903254
 0.1735252776717331

In [60]:
# Матрица из собственных векторов по столбцам imag от третьего вектора, real от 1 и 2

ErrorException: syntax: missing comma or ) in argument list

In [7]:
ds = CoupledODEs(TM, shift, p, diffeq = integ_set)
sol, trange = trajectory(ds, t, Ttr = tt, Δt = tstep)
println("Length sol: $(length(sol))");

Length sol: 1500001


In [8]:
lb_size_ts = 30; tck_size_ts = 25
width = 2000; height = 800;

In [31]:
function global_max(timesereis, ϵ = 1e-2)

    tr, trange = timeseries
    glob_maxs, trange_glob_maxs = Float64[], Float64[]
    real_global_max =floor(maximum(tr), digits = 3)
    idxs = Int64[]
    
    for idx in range(2, length(tr) - 1, step = 1)
        if tr[idx-1] < tr[idx] > tr[idx+1]
            if abs(tr[idx] -real_global_max) < ϵ
                push!(glob_maxs, tr[idx])
                push!(trange_glob_maxs, trange[idx])
                push!(idxs, idx)
            end
        end
    end

    return glob_maxs,trange_glob_maxs, idxs
end
function interval_turns(timeseries, index, fp)

    tr, trange = timeseries
    
    flag = false
    timelabel = 0
    
    while flag == false
        index+=1
        if sol[index, 3] <= fp[1][3]
            timelabel = index - 1
            break
        end
    end
    return timelabel
end
function local_max(sol, index_global_max, timelabel)
   
    index = index_global_max
    local_maxs, trange_local_maxs = Float64[], Float64[]
    idxs = Int64[]
    
    while index <= timelabel

        if sol[index-1, 1] < sol[index, 1] > sol[index+1, 1]
            push!(local_maxs, sol[index, 1])
            push!(trange_local_maxs, trange[index])
            push!(idxs, index)
        end

        index+=1
    end
    index-=1

    return  local_maxs,trange_local_maxs, idxs
end

function check_hom(sol, fp, ϵ = 1e-3)
    
    for i in range(1, length(sol), step = 1)
        if distance(sol[i], fp[1]) <= ϵ
            return true
            break
        end

    end
    return false
end

check_hom (generic function with 2 methods)

In [32]:
function distance(pointA, pointB)
    return sqrt( (pointA[1] - pointB[1])^2 + (pointA[2] - pointB[2])^2 + (pointA[3] - pointB[3])^2 )
end

distance (generic function with 1 method)

In [33]:
check_hom(sol, fp)

false

In [10]:
timeseries = [sol[:, 3], trange]

2-element Vector{AbstractVector{Float64}}:
 [0.5600016673260375, 0.5598419338452583, 0.5596824163772047, 0.5595231214957459, 0.5593640558600986, 0.5592052262179555, 0.5590466394087509, 0.5588883023661656, 0.5587302221208549, 0.5585724058026897  …  0.5067984736353885, 0.506937931208998, 0.5070773407184594, 0.5072167021588159, 0.5073560155250357, 0.507495280812546, 0.5076344980167194, 0.5077736671333908, 0.5079127881583257, 0.5080518610877389]
 1500.0:0.001:3000.0

In [104]:
glob_maxs , trange_glob_maxs, indexs_global_max = global_max(timeseries, 1e-2)

([0.5723694814219829, 0.5753400594507811, 0.5723694826537408, 0.5753400603766905, 0.5723694808266685, 0.5753400582564042, 0.5723694777036131, 0.5753400560182861, 0.572369481684595, 0.5753400596820257  …  0.5723694792633949, 0.5753400574784475, 0.572369482297593, 0.5753400601993286, 0.5723694822739487, 0.5753400598750134, 0.5723694791907828, 0.5753400565038161, 0.5723694797075575, 0.5753400578904609], [500.8336, 505.1084, 509.336, 513.6108, 517.8384, 522.1132, 526.3407, 530.6155, 534.8431, 539.1179  …  959.9618, 964.2366, 968.4642, 972.739, 976.9666, 981.2414, 985.469, 989.7438, 993.9713, 998.2461], [8337, 51085, 93361, 136109, 178385, 221133, 263408, 306156, 348432, 391180  …  4599619, 4642367, 4684643, 4727391, 4769667, 4812415, 4854691, 4897439, 4939714, 4982462])

In [ ]:
number_global_max = 40
index_global_max = indexs_global_max[number_global_max]
timelabel = interval_turns(timeseries, index_global_max, fp)

In [ ]:
local_maxs,trange_local_maxs, idxs = local_max(sol, index_global_max, timelabel)
println("Number of turns: $(length(local_maxs) - 1)")

In [105]:
count_global_max = length(indexs_global_max)-1
range_global_max = range(1, count_global_max, step = 1)
max_count_turns = Int64[]

Int64[]

In [106]:
for i in range_global_max
    index_global_max = indexs_global_max[i]
    timelabel = interval_turns(timeseries, index_global_max, fp)
    local_maxs,trange_local_maxs, idxs = local_max(sol, index_global_max, timelabel)
    push!(max_count_turns, length(local_maxs) - 1)
end

In [107]:
maximum(max_count_turns)

4

In [32]:
ts, tf = 1,200000

f = Figure(resolution = (width, height))
axisE = Axis(f[1, 1], xlabel = L"$time$", ylabel = L"$E$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

axisy = Axis(f[2, 1], xlabel = L"$time$", ylabel = L"$y$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

lines!(axisE, trange[ts:tf], sol[ts:tf, 1], linewidth = 1.5, color = :deeppink)
lines!(axisy, trange[ts:tf], sol[ts:tf, 3], linewidth = 1.5, color = :deeppink)
hlines!(axisy, fp[1][3], color = :green)
hlines!(axisE, fp[1][1], color = :green)

display(GLMakie.Screen(), f)
#display(f)

GLMakie.Screen(...)

In [92]:
length(sol)

15000001

In [34]:
ts, tf = 1, 1500000
idx, idy, idz = 2, 3, 1
dis  = 60;

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

lines!(sol[ts:tf, idx], sol[ts:tf, idy], sol[ts:tf, idz], linewidth = 1.0, color = :black)

for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [40]:
ts, tf = index_global_max, timelabel
#tscon, tfcon = timelabel, timelabel + 100
idx, idy, idz = 2, 3, 1
dis  = 60;

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
    
scale!(ax.scene, 50, 50, 1)

lines!(sol[ts:tf, idx], sol[ts:tf, idy], sol[ts:tf, idz], linewidth = 1.0, color = :black)

lines!(sol[index_global_max:idxs[2], idx], sol[index_global_max:idxs[2], idy], sol[index_global_max:idxs[2], idz], linewidth = 1.0, color = :green)
lines!(sol[idxs[2]:idxs[3], idx], sol[idxs[2]:idxs[3], idy], sol[idxs[2]:idxs[3], idz], linewidth = 1.0, color = :orange)
lines!(sol[idxs[3]:idxs[4], idx], sol[idxs[3]:idxs[4], idy], sol[idxs[3]:idxs[4], idz], linewidth = 1.0, color = :deeppink)
lines!(sol[idxs[4]:idxs[5], idx], sol[idxs[4]:idxs[5], idy], sol[idxs[4]:idxs[5], idz], linewidth = 1.0, color = :lime)
#lines!(sol[idxs[5]:idxs[6], idx], sol[idxs[5]:idxs[6], idy], sol[idxs[5]:idxs[6], idz], linewidth = 1.0, color = :magenta)




for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 8)
end

scatter!(ax, sol[indexs_global_max[number_global_max], idx], sol[indexs_global_max[number_global_max], idy], sol[indexs_global_max[number_global_max], idz], color = :green, markersize = 8)

for id in idxs    
    scatter!(ax, sol[id, idx], sol[id, idy], sol[id, idz], color = :red, markersize = 8 )
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)